# Programming Homework 1

## Instructions

- Do not import other libraries. You are only allowed to use Math, Numpy, Scipy packages which are already imported in the file.
- Please follow the type annotations. There are some type annotations of the parameters of function calls and return values. Please use Python 3.5 or 3.6 (for full support of typing annotations). You can use Numpy/Scipy inside the function.  You have to make the functions’ return values match the required type.
- In this programming assignment you will to implement **k-Nearest Neighbours and Decision Tree**. We provide the bootstrap code and you are expected to complete the **classes** and **functions**.
- Download all files of PA1 from Vocareum and save in the same folder.
- Only modifications in files {`hw1_knn.py`, `hw1_dt.py`, `utils.py`} will be accepted and graded. All other modifications will be ignored. Submit those three files on Vocareum once you have finished. Which means you need to delete unnecessary files before you submit your work on Vocareum.

## Office Hour:
```
Week 2
Jan. 14th Monday	LVL 2nd Floor-201B	1:00pm to 3:00pm	Cheng-Ju Lin chengjul@usc.edu
Jan. 15th Tuesday	LVL 2nd Floor-201B	1:00pm to 3:00pm	Yang Fang yangfang@usc.edu
Jan. 17th Thursday	LVL 2nd Floor-202B	10:00am to 12:00pm	Yixian Di yixiandi@usc.edu
Week 3
Jan. 22th Tuesday	SAL 125         	11:00am to 1:00pm	Ashir Alam ashirala@usc.edu
Jan. 23th Wednesday	SAL 125         	11:00am to 1:00pm	Ashir Alam ashirala@usc.edu
Jan. 24th Thursday	LVL 2nd Floor-202B	10:00am to 12:00pm	Yixian Di yixiandi@usc.edu
```

## Problem 1: K-nearest neighbor (KNN) for binary classification (50 points)

#### Some notes

In this task, we will use four distance functions: (we removed the vector symbol for simplicity)

- Euclidean distance:  $$d(x, y) = \sqrt{\langle x - y, x - y \rangle}$$
- Inner product distance: $$d(x, y ) = \langle x, y \rangle$$
- Gaussian kernel distance: 
    $$d(x, y ) = - \exp({−\frac 12 \langle x - y, x - y \rangle}) $$
- Cosine Distance: $$d(x, y) =\cos(\theta )={\mathbf {x} \cdot \mathbf {y}  \over \|\mathbf {x} \|\|\mathbf {y} \|}$$

F1-score is a important metric for binary classification, as sometimes the accuracy metric has the false positive (a good example is in MLAPP book 2.2.3.1 “Example: medical diagnosis”, Page 29).
We have provided a basic definition. For more you can read 5.7.2.3 from MLAPP book.

<img src="F1Score.png">


### Part 1.1 Distance Functions

Implement the class in file *hw1_knn.py*
    - KNN
    
and the functions in *utils.py*    
    - f1_score
    - euclidean_distance
    - inner_product_distance
    - gaussian_kernel_distance
    - cosine distance

In [126]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
import numpy as np
from hw1_knn import KNN
from utils import euclidean_distance, gaussian_kernel_distance, inner_product_distance, cosine_sim_distance
from utils import f1_score
distance_funcs = {
    'euclidean': euclidean_distance,
    'gaussian': gaussian_kernel_distance,
    'inner_prod': inner_product_distance,
    'cosine_dist': cosine_sim_distance,
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Data processing 

Do the following steps:

- Load data (features and values) from function generate data_processing
- Check that there are 303 data samples and each sample have a feature vector of length 14.
- Split the whole data set into three parts:
     - the train set contains 80% samples,
     - the validation set contains the next 15% samples,
     - the test set contains the rest 5% samples.

In [127]:
from data import data_processing
Xtrain, ytrain, Xval, yval, Xtest, ytest = data_processing()

Xtrain, ytrain

(array([[ 1., 52.,  1., ...,  1.,  3.,  7.],
        [ 1., 48.,  1., ...,  1.,  2.,  7.],
        [ 1., 58.,  0., ...,  1.,  2.,  3.],
        ...,
        [ 1., 41.,  1., ...,  1.,  0.,  7.],
        [ 1., 62.,  1., ...,  2.,  2.,  7.],
        [ 1., 45.,  1., ...,  2.,  0.,  7.]]),
 array([0., 1., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1.,
        0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1.,
        1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0.,
        0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0.,
        0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
        1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 1.,

#### Model selection 
In kNN model, the parameter k is a hyper-parameter. In this task, we search for the best k.

In [128]:
best_k, best_model = KNN.model_selection_without_normalization(distance_funcs, Xtrain, ytrain, f1_score, Xval, yval, Xtest, ytest)

[part 1.1] euclidean	k: 1	train: 0.99038	valid: 0.60870
[part 1.1] euclidean	k: 2	train: 0.79848	valid: 0.80000
[part 1.1] euclidean	k: 3	train: 0.72727	valid: 0.76596
[part 1.1] euclidean	k: 4	train: 0.74790	valid: 0.79245
[part 1.1] euclidean	k: 5	train: 0.68966	valid: 0.68085
[part 1.1] euclidean	k: 6	train: 0.70690	valid: 0.75472
[part 1.1] euclidean	k: 7	train: 0.66332	valid: 0.72000
[part 1.1] euclidean	k: 8	train: 0.67890	valid: 0.74510
[part 1.1] euclidean	k: 9	train: 0.65263	valid: 0.72340
[part 1.1] euclidean	k: 10	train: 0.66359	valid: 0.78431
[part 1.1] gaussian	k: 1	train: 0.99038	valid: 0.60870
[part 1.1] gaussian	k: 2	train: 0.79245	valid: 0.80000
[part 1.1] gaussian	k: 3	train: 0.71770	valid: 0.76596
[part 1.1] gaussian	k: 4	train: 0.74074	valid: 0.81481
[part 1.1] gaussian	k: 5	train: 0.68627	valid: 0.68085
[part 1.1] gaussian	k: 6	train: 0.69748	valid: 0.75472
[part 1.1] gaussian	k: 7	train: 0.66010	valid: 0.72000
[part 1.1] gaussian	k: 8	train: 0.66667	valid: 0.74510

#### Classification
Here we are trying to do classification on another test set. You can make up your own data. However, we will test it on another dataset.

In [129]:
KNN.test_classify(best_model)

IndexError: index 14 is out of bounds for axis 0 with size 14

### Part 1.2 Data transformation

We are going to add one more step (data transformation) in the data processing part and see how it works. 
Sometimes, normalization plays an important role to make a machine learning model work (check term “Feature scaling” in wiki).

Here, we take two different data transformation approaches.

#### Normalizing the feature vector 

This one is simple but some times may work well. Given a feature vector $x$, the normalized feature vector is given by 

$$ x' = \frac x {\sqrt{\langle x, x \rangle}} $$
If a vector is a all-zero vector, we let the normalized vector also be a all-zero vector.


#### Min-max scaling the feature matrix

The above normalization is data independent, that is to say, the output of the normalization function doesn’t depend on the rest training data. However, sometimes it would be helpful to do data dependent normalization. One thing to note is that, when doing data dependent normalization, we can only use training data, as the test data is assumed to be unknown during training (at least for most classification tasks).

The min-max scaling works as follows: after min-max scaling, all values of training data’s feature vectors are in the given range.
Note that this doesn’t mean the values of the validation/test data’s fea- tures are all in that range, because the validation/test data may have dif- ferent distribution as the training data.

**Implement** the functions in *utils.py*    
    - normalize
    - min_max_scale

In [65]:
from utils import NormalizationScaler, MinMaxScaler

scaling_classes = {
    'min_max_scale': MinMaxScaler,
    'normalize': NormalizationScaler,
}

#### Model selection

Repeat the model selection part in part 1.2.

In [66]:
best_k, best_model = KNN.model_selection_with_transformation(distance_funcs,scaling_classes, Xtrain, ytrain, f1_score, Xval, yval, Xtest, ytest)

[part 1.2] euclidean	min_max_scale	k: 1	train: 0.00000	valid: 0.00000
[part 1.2] euclidean	min_max_scale	k: 2	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 3	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 4	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 5	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 6	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 7	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 8	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 9	train: 0.62500	valid: 0.67647
[part 1.2] euclidean	min_max_scale	k: 10	train: 0.62500	valid: 0.67647
[part 1.2] gaussian	min_max_scale	k: 1	train: 0.00000	valid: 0.00000
[part 1.2] gaussian	min_max_scale	k: 2	train: 0.62500	valid: 0.67647
[part 1.2] gaussian	min_max_scale	k: 3	train: 0.62500	valid: 0.67647
[part 1.2] gaussian	min_max_scale	k: 4	train: 0.62500	valid: 0.67647
[part 1.2] gaussian	min

## Grading Guideline for KNN
1. UTILS function: 15 points <br>

2. 2 functions in hw1_Knn (10 points- 5 each) <br>

3. Finding best K before scaling - 10 points <br>

4. Finding best K after scaling - 10 points <br>

5. Doing classification of the data - 5 points <br>

# Problem 2: Decision Tree (50 points)
- Remember from lecture, we learned that we can use decision tree to solve classification and regression problem. Mostly we focus on classification.
- In problem 1 we used KNN to do classification. We could use decision tree algorithm to do the same job.
- For Decision Tree, we will implement ID3 algorithm. It's garanteed that all features are discrete.
## Part 2.1 Implementation
### 2.1.1
- In ID3 algorithm, we use Entropy to measure the uncertainty in the data set. We use Information Gain to measure the quality of a split.
- Entropy: H(S)=\\(\sum_{x∈X} -p(x)log_2p(x)\\)
- Information_Gain: IG(S,A) = H(S)-\\(\sum_{t∈T}p(t)H(T)\\) = H(S) - H(S|A)
- see more detail on [ID3 Algorithm](https://en.wikipedia.org/wiki/ID3_algorithm)
In this section, you need to implement Information_Gain function on utils.py.
```
def Information_Gain(branches):
# calculate information_gain according to branches seperated by one feature
# input:
    -branches: List[List[int]] for a specific attribute, number of cases belongs to each attribut value and class, num_attribute_values*num_classes
# return: float
```
### 2.1.2 
- In ID3 algorithm, we use the largest information gain to split the set S.
- Here is the pseudo code of ID3 algorithm.
```
Algorithm 3 The recursive procedure of decision tree algorithm
function TREENODE.SPLIT(self)
    ﬁnd the best attribute to split the node
    split the node into child nodes
    for child node in child nodes do
        if child node.splittable then .
            child node.split()
        end if
    end for
end function
```
- Implement TreeNode split function and TreeNode predict function in hw1_dt.py:
    - TreeNode.split<br>
    **Note: when there is a tie of information gain, always choose the attribute which has more attribute values. If they are all same, use the one with small index. Also build your child list with increasing order of attribute value.**
    - TreeNode.predict
```
def TreeNode.split()
# check if current node is splitable, try to split it with all possible features
def TreeNode.predict()
# predic according to current node:
# if leaf node: return current leaf node label
# if non-leaf node: split it to child node
```
- Implement Decision Tree predict and train function in hw1_dt.py:
    - DecisionTree.train
    - DecisionTree.predict

```
def DecisionTree.train(features, labels)
# train decision tree based on training dataset, grow your decision tree from root node
# input: 
    - features: List[List[any]] traning data, num_cases*num_attributes
    - labels: List[any] traning labels, num_cases*1
```

## Part 2.2 Sanity Test
Do the following steps, as a simple test to check your algorithm works well
- Load training data (features and values) from function data.sample_decision_tree_data.
- Create a Decision Tree based on training data.
- Load test data from function data.sample_decision_tree_test.
- Test the prediction function of your algorithm.

In [8]:
import data
import hw1_dt as decision_tree
import utils as Utils
from sklearn.metrics import accuracy_score

features, labels = data.sample_decision_tree_data()

# build the tree
dTree = decision_tree.DecisionTree()
dTree.train(features, labels)

# print
Utils.print_tree(dTree)

branch 0{
	deep: 0
	num of samples for each class: 2 : 2 
	split by dim 0
	branch 0->0{
		deep: 1
		num of samples for each class: 1 : 0 
		class: 0
	}
	branch 0->1{
		deep: 1
		num of samples for each class: 1 : 1 
		split by dim 1
		branch 0->1->0{
			deep: 2
			num of samples for each class: 1 : 0 
			class: 0
		}
		branch 0->1->1{
			deep: 2
			num of samples for each class: 0 : 1 
			class: 1
		}
	}
	branch 0->2{
		deep: 1
		num of samples for each class: 0 : 1 
		class: 1
	}
}


In [9]:
# data
X_test, y_test = data.sample_decision_tree_test()

# testing
y_est_test = dTree.predict(X_test)
test_accu = accuracy_score(y_est_test, y_test)
print('test_accu', test_accu)

ValueError: 'b' is not in list

## Part 2.3 Train and Predict
### 2.3.1
- Load data (features and values) from function data.load_decision_tree_data.
- Train your decision tree

In [10]:
#load data
X_train, y_train, X_test, y_test = data.load_decision_tree_data()

# set classifier
dTree = decision_tree.DecisionTree()

# training
dTree.train(X_train.tolist(), y_train.tolist())

IndexError: index 3 is out of bounds for axis 0 with size 3

### 2.3.2
- Print your decision tree.

In [4]:
# print
Utils.print_tree(dTree)

branch 0{
	deep: 0
	num of samples for each class: 845 : 260 : 0 


TypeError: unsupported format string passed to NoneType.__format__

### 2.3.3
- do prediction on test dataset.

In [5]:
import json
# testing
y_est_test = dTree.predict(X_test)
test_accu = accuracy_score(y_est_test, y_test)
print('test_accu', test_accu)

AttributeError: 'NoneType' object has no attribute 'index'

## Part 2.4 Pruning The Tree
Sometimes, in order to prevent overfitting. We need to pruning our Decition Tree. There are several approaches to avoiding overfitting in building decision trees. 

- Pre-pruning that stop growing the tree earlier, before it perfectly classifies the training set.
- Post-pruning that allows the tree to perfectly classify the training set, and then post prune the tree. 

Practically, the second approach of post-pruning overfit trees is more successful because it is not easy to precisely estimate when to stop growing the tree.
We will use Reduced Error Pruning, as one of Post-pruning in this part.
```
Reduced Error Pruning
0. Split data into training and validation sets.
1. Do until further pruning is harmful:
2. Evaluate impact on validation set of pruning each possible node (plus those below it)
3. Greedily remove the one that most improves validation set accuracy
- Produces smallest version of most accurate subtree.
- Requires that a lot of data be available.
```
For Pruning of Decision Tree, you can refer [Reduce Error Pruning](http://jmvidal.cse.sc.edu/talks/decisiontrees/reducederrorprun.html?style=White) and P69 of Textbook: Machine Learning -Tom Mitchell.

### 2.4.1 
**Hint: in this part, you can add another parameters or functions in TreeNode class and DecisionTree class for your convenience. But your changes should not influent results of previous parts.**<br>
implement the reduced_error_pruning function on util.py.

```
def reduced_error_pruning(decitionTree):
# input: 
    - decitionTree: decitionTree trained based on training data set.
    - X_test: List[List[any]] test data, num_cases*num_attributes
    - y_test: List[any] test labels, num_cases*1
```

In [ ]:
Utils.reduced_error_pruning(dTree, X_test, y_test)

### 2.4.2
Test your prediction accuracy on validation dataset after pruning.

In [ ]:
y_est_test = dTree.predict(X_test)
test_accu = accuracy_score(y_est_test, y_test)
print('test_accu', test_accu)

### 2.4.3
Print your decision tree after pruning.

In [ ]:
Utils.print_tree(dTree)

## Grading Guidline
1. Information_Gain function - 10 points <br>
we will test your Infomation Gain function on another ten inputs. To receive full credits of this part, your function should be able to output right valus.
2. Train your decision tree - 15 points <br>
we will test your decision tree after training on training dataset. To receive full credit of this part, your algorithm will generate the identical decision tree as our answer.
3. Prediction of decision tree - 10 points <br>
we will use another dataset to test your prediction part of decision tree, you can assume that test dataset has identical attributs and values as traning dataset. To receive full credit of this part, your algorithm will generate the identical prediction of our answer.
4. Pruning of decision tree - 15 points <br>
we will test your decision tree after pruning. To receive full credit of this part, your algorithm will generate the identical decision tree as our answer.

Good Luck! : )